In [31]:
import plotly.express as px
import pandas as pd
basepath = 'your_path_here'

In [32]:
dfw_prop_df = pd.read_pickle(basepath + "Data/OnTime/PropagateSummary_DFW_OUT.pkl")

In [33]:
dfw_1 = dfw_prop_df[dfw_prop_df['Flights']== 1]
dfw_gt_1 = dfw_prop_df[dfw_prop_df['Flights'] > 1]

prop_cnts_by_dest = pd.DataFrame()
dfw_le_prop_tn = dfw_1[dfw_1['PropagateCount'] == 0]
dfw_le_prop_tn["Flights"] = dfw_le_prop_tn['Flights']
dfw_le_prop_tn["PropagateCount"] = dfw_le_prop_tn['PropagateCount']
dfw_le_prop_tn["P_Minutes"] = dfw_le_prop_tn['List_of_PDelay_Minutes'].apply(lambda x : int(float(x))).copy()
dfw_le_prop_tn.insert(0, 'Propagation Number', 0)
prop_cnts_by_dest = pd.concat([prop_cnts_by_dest, dfw_le_prop_tn[['Propagation Number', 'Flights', 'P_Minutes']]], axis= 0)

for idx, row in dfw_gt_1.iterrows():
    for prop in range(1, row['PropagateCount'] + 1,1):
        row_dict = {}
        if row['PropagateCount'] >= prop:
            row_dict['Propagation Number'] = prop
            row_dict['Flights'] = row['Flights']
            row_dict['PropagateCount'] = row['PropagateCount']
            row_dict['P_Minutes'] = int(float(row['List_of_PDelay_Minutes'][1:-1].split(",")[prop]))
            row_df = pd.DataFrame.from_dict([row_dict])
            prop_cnts_by_dest = pd.concat([prop_cnts_by_dest, row_df], axis= 0)

prop_cnts_by_dest.sort_values(by= ['Propagation Number'], ascending= [True], inplace= True)
prop_cnts_by_dest.reset_index(drop= True, inplace= True)


In [34]:
dfw_Dest_cnts = prop_cnts_by_dest.groupby(['Propagation Number']).agg({"P_Minutes": "mean", "Flights": "sum"}).rename(columns={"P_Minutes":'Average Delay'})
dfw_Dest_cnts.reset_index(inplace=True)

In [40]:
fig = px.bar(data_frame= dfw_Dest_cnts, x= 'Propagation Number', y= 'Average Delay', text= 'Flights', height = 600, width = 600)
fig.add_annotation(x=2, y=90, text= "Number of flights inside each bar", showarrow=False)
fig.add_annotation(x=2, y=85, text= "First bar is Average Weather Delay", showarrow=False)
fig.update_yaxes(showgrid=False)
fig.write_html(basepath + "Code/VisualWork/Avg_DelMinsPerLeg.html")
fig.write_image(basepath + "Code/VisualWork/Avg_DelMinsPerLeg.png")
fig